<h1>Imports</h1>

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import face_recognition
import pandas as pd
import subprocess
import urllib.request
from pytube import YouTube
import os
from youtube_transcript_api import YouTubeTranscriptApi

<h1>Function declaration</h1>

In [ ]:
# Function to calculate speaker position relative to the center
def calculate_speaker_position(frame, shoulder_midpoint_x):
    frame_width = frame.shape[1]
    speaker_position = shoulder_midpoint_x - frame_width / 2
    return speaker_position

<h1>Process Each Frame</h1>

In [ ]:
pip install yt-dlp moviepy opencv-python


In [ ]:
pip install pafy opencv-python


In [ ]:

# updated file
import os
import cv2
from moviepy.editor import VideoFileClip
from yt_dlp import YoutubeDL
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
mp_drawing = mp.solutions.drawing_utils

# excel_file = 'TED_Dataset.xlsx'
# sheet_name = 'TED_Dataset'
# df = pd.read_excel(excel_file, sheet_name=sheet_name)
# video_links = df['youtube_video_code']
# Load the video
for video_link in range(1):
    youtube_url = "https://youtu.be/0EykDvxJAyA?si=mbwfqH0E1Qp1TAJe"

    # Download the video using yt-dlp
    ydl_opts = {
        'format': 'best',
        'outtmpl': 'downloaded_video.%(ext)s',
    }

    with YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        video_file = ydl.prepare_filename(info_dict)

    # Load the video using moviepy
    clip = VideoFileClip(video_file)

    # Open the video file using OpenCV
    cap = cv2.VideoCapture(video_file)

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    frame_count = int(cap.get(7))
    fps = int(cap.get(5))
    video_length_seconds = frame_count / fps

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Initialize MediaPipe holistics
    mp_holistic = mp.solutions.holistic
    holistic_model = mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    # Load pre-trained emotion detection model
    emotion_model = load_model('emotion_model.h5')  # Load your emotion detection model here

    # Create empty lists to store data for graphs
    shoulder_midpoints = []
    head_turn_angles_right = []
    head_turn_angles_left = []
    emotions = []  # List to store detected emotions
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprised', 'Neutral']
    left_hand = []
    right_hand = []

    # Set the graph display parameters
    graph_height = 200  # Height of the graph (increased)
    graph_color = (0, 0, 0)  # Color of the graph (BGR format)
    line_width = 1  # Width of the plotted line

    # Set video dimensions
    output_width = 800  # Adjust the width as needed
    output_height = int(frame_height * (output_width / frame_width))

    # Create an output video writer
    output_video = 'output_video.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (output_width, output_height))  # Adjust dimensions

    # Initialize frame index
    frame_index = 0
    frame_counter = 0
    # Process each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_counter += 1
        # Convert the frame to RGB (MediaPipe requires RGB input)
        if frame_counter >= fps/6:
            frame_counter = 0
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_haar_cascade.detectMultiScale(frame_grey)

            # Detect emotions in the frame and append to the emotions list
            try:
                for t in range(1):
                    for (x, y, w, h) in faces:
                        # Frame rectangle
                        cv2.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 0, 255), thickness=2)
                        roi_gray = frame_grey[y - 5:y + h + 5, x - 5:x + w + 5]
                        roi_gray = cv2.resize(roi_gray, (48, 48))
                        image_pixels = img_to_array(roi_gray)
                        image_pixels = np.expand_dims(image_pixels, axis=0)
                        # Normalize
                        image_pixels /= 255
                        predictions = emotion_model.predict(image_pixels)
                        max_index = np.argmax(predictions[0])
                        detected_emotion = emotion_labels[max_index]
                        emotions.append(detected_emotion)
                        print(emotions)
                        print("\n")
            except:
                emotions.append(None)

            # Detect pose in the frame
            results = pose.process(frame_rgb)

            if results.pose_landmarks is not None:
                # Extract landmarks of the left and right shoulders (ids 11 and 12)
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

                if left_shoulder and right_shoulder:
                    # Calculate the midpoint of the shoulders
                    shoulder_midpoint_x = (left_shoulder.x + right_shoulder.x) / 2

                    # Append the shoulder midpoint to the list
                    shoulder_midpoints.append(shoulder_midpoint_x)
                else:
                    # Append None for frames with no shoulder points
                    shoulder_midpoints.append(None)
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Extract landmarks of the left and right eyes and nose
                left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE_INNER]
                right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE_INNER]
                nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

                if left_eye and right_eye and nose:
                    # Calculate the angle between the line connecting the eyes and a horizontal reference line
                    eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
                    eye_left_nose_vector = np.array([nose.x-left_eye.x ,  nose.y-left_eye.y])
                    eye_right_nose_vector = np.array([right_eye.x - nose.x, right_eye.y - nose.y])

                    dot_product_left = np.dot(eye_line_vector, eye_left_nose_vector)
                    eye_line_magnitude = np.linalg.norm(eye_line_vector)
                    eye_left_nose_magnitude = np.linalg.norm(eye_left_nose_vector)

                    dot_product_right = np.dot(eye_line_vector, eye_right_nose_vector)
                    eye_right_nose_magnitude = np.linalg.norm(eye_right_nose_vector)

                    cosine_angle_left = dot_product_left / (eye_line_magnitude * eye_left_nose_magnitude)
                    cosine_angle_right = dot_product_right / (eye_line_magnitude * eye_right_nose_magnitude)

                    head_turn_angle_left = np.arccos(cosine_angle_left) * (180 / np.pi)
                    head_turn_angle_right = np.arccos(cosine_angle_right) * (180 / np.pi)

                    head_turn_angles_left.append(head_turn_angle_left)
                    head_turn_angles_right.append(head_turn_angle_right)

                else:
                    head_turn_angles_left.append(None)
                    head_turn_angles_right.append(None)
            else:
                shoulder_midpoints.append(None)
                head_turn_angles_left.append(None)
                head_turn_angles_right.append(None)

            # Update the frame index
            frame_index += 1

            # Check if the shoulder_midpoints list is not empty
            if shoulder_midpoints:
                # Filter out None values and calculate the minimum and maximum values for each graph
                filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
                if filtered_shoulder_midpoints:
                    min_shoulder_midpoint = min(filtered_shoulder_midpoints)
                    max_shoulder_midpoint = max(filtered_shoulder_midpoints)
                else:
                    # If the list is empty, set min and max to 0
                    min_shoulder_midpoint = 0
                    max_shoulder_midpoint = 0
            else:
                # If the list is empty, set min and max to 0
                min_shoulder_midpoint = 0
                max_shoulder_midpoint = 0

            if head_turn_angles_left:
                # Filter out None values and calculate the minimum and maximum values for each graph
                filtered_head_turn_angles_left = [angle for angle in head_turn_angles_left if angle is not None]
                if filtered_head_turn_angles_left:
                    min_head_turn_angle_left = min(filtered_head_turn_angles_left)
                    max_head_turn_angle_left = max(filtered_head_turn_angles_left)
                else:
                    min_head_turn_angle_left = 0
                    max_head_turn_angle_left = 0
                
            else:
                min_head_turn_angle_left = 0
                max_head_turn_angle_left = 0
            
            if head_turn_angles_right:
                # Filter out None values and calculate the minimum and maximum values for each graph
                filtered_head_turn_angles_right = [angle for angle in head_turn_angles_right if angle is not None]
                if filtered_head_turn_angles_right:
                    min_head_turn_angle_right = min(filtered_head_turn_angles_right)
                    max_head_turn_angle_right = max(filtered_head_turn_angles_right)
                else:
                    min_head_turn_angle_right = 0
                    max_head_turn_angle_right = 0
            else:
                min_head_turn_angle_right = 0
                max_head_turn_angle_right = 0
            
            frame_rgb.flags.writeable = False
            hand_results = holistic_model.process(frame_rgb)
            frame_rgb.flags.writeable = True

            left_hand_sum_distance = 0
            if hand_results.left_hand_landmarks is not None:
                # Drawing Left hand Land Marks
                # mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                mp_drawing.draw_landmarks( frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                left_hand_landmarks = hand_results.left_hand_landmarks.landmark
                for landmark in left_hand_landmarks:
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    distance = np.sqrt(x**2 + y**2 + z**2)
                    left_hand_sum_distance += distance
                    
                left_hand.append(left_hand_sum_distance)
            else:
                left_hand.append(None)
            
            right_hand_sum_distance = 0
            if hand_results.right_hand_landmarks is not None:
                mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS )
                right_hand_landmarks = hand_results.right_hand_landmarks.landmark
                for landmark in right_hand_landmarks:
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    distance = np.sqrt(x**2 + y**2 + z**2)
                    right_hand_sum_distance += distance
                right_hand.append(right_hand_sum_distance)
            else:
                right_hand.append(None)

            filtered_left_hand = [point for point in left_hand if point is not None]
            if filtered_left_hand:
                # Filter out None values and calculate the minimum and maximum values for each graph
                min_left_hand = min(filtered_left_hand)
                max_left_hand = max(filtered_left_hand)
            else:
                # If the list is empty, set min and max to 0
                min_left_hand = 0
                max_left_hand = 0

            filtered_right_hand = [point for point in right_hand if point is not None]
            if filtered_right_hand:
                # Filter out None values and calculate the minimum and maximum values for each graph
                min_right_hand = min(filtered_right_hand)
                max_right_hand = max(filtered_right_hand)
            else:
                # If the list is empty, set min and max to 0
                min_right_hand = 0
                max_right_hand = 0


            # Create separate figures for each graph
            plt.figure(figsize=(output_width / 100, 3 * graph_height / 100))  # Adjust graph size

            # Shoulder Movement Graph
            plt.subplot(4, 1, 1)
            plt.plot(np.arange(frame_index), shoulder_midpoints, color='red', linewidth=line_width, marker='o', markersize=1)
            plt.ylim([min_shoulder_midpoint, max_shoulder_midpoint])
            plt.xlim([0, video_length_seconds*6])
            plt.axis('on')
            plt.ylabel('Shoulder Midpoint', fontsize=12)

            # Head Turn Angle Graph
            plt.subplot(4, 1, 2)
            plt.plot(np.arange(frame_index), head_turn_angles_left, color='red', linewidth=line_width, marker='o', markersize=1, label='Left Turn Angle')
            plt.plot(np.arange(frame_index), head_turn_angles_right, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right Turn Angle')
            plt.ylim([min(min_head_turn_angle_left, min_head_turn_angle_right), max(max_head_turn_angle_left, max_head_turn_angle_right)])
            plt.xlim([0, video_length_seconds*6])
            plt.axis('on')
            plt.ylabel('Head Turn Angle (degrees)', fontsize=12)
            plt.legend()

            # Emotion Graph
            plt.subplot(4, 1, 3)
            x_values = np.arange(len(emotions))  # Use the length of the emotions list as x-axis values
            plt.scatter(x_values, [emotion_labels.index(e) if e is not None else -1 for e in emotions],
                        color='green', s=5)  # Plot emotions as dots
            plt.ylim(-1, len(emotion_labels))  # Set y-axis limits
            plt.xlim(0, video_length_seconds*6)  # Set x-axis limits
            plt.yticks(range(len(emotion_labels)), emotion_labels)  # Set y-tick labels
            plt.grid(True, linestyle='--', alpha=0.6)  # Add gridlines
            plt.ylabel('Emotion', fontsize=12)  # Add y-axis label

            # Hand Graph
            plt.subplot(4, 1, 4)
            plt.plot(np.arange(frame_index), left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
            plt.plot(np.arange(frame_index), right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
            plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
            plt.xlim([0, video_length_seconds*6])
            plt.axis('on')
            plt.ylabel('Hand movements', fontsize=12)
            plt.legend()
            
            plt.tight_layout()

            # Save the graph with labels
            plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
            plt.clf()

            graph = cv2.imread('temp_graph.png')

            # Resize the graph to match the video width and increased height
            graph = cv2.resize(graph, (output_width, 3 * graph_height))

            # Display the input video
            cv2.imshow('Input Video', frame)

            # Display the graph with labels
            cv2.imshow('Graph', graph)

            # Write the frame with graph to the output video
            out.write(frame)

        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()
